In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'd:\\kidney-disease-classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney-data", "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone","CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config
        self.validation_ratio = 0.2


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train_valid_generator(self):
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            labels="inferred",
            label_mode="categorical",
            class_names=['Cyst', 'Normal', 'Stone', 'Tumor'],
            color_mode="rgb",
            batch_size=self.config.params_batch_size,
            image_size=self.config.params_image_size[:-1],
            shuffle=True,
            seed=69,
            validation_split=self.validation_ratio,
            subset="training",
            interpolation="bilinear",
            verbose=True,
        )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            labels="inferred",
            label_mode="categorical",
            class_names=['Cyst', 'Normal', 'Stone', 'Tumor'],
            color_mode="rgb",
            batch_size=self.config.params_batch_size,
            image_size=self.config.params_image_size[:-1],
            shuffle=True,
            seed=69,
            validation_split=self.validation_ratio,
            subset="validation",
            interpolation="bilinear",
            verbose=True,
        )


    def train(self):
        self.steps_per_epoch = int(1200*(1-self.validation_ratio) // self.config.params_batch_size)
        self.validation_steps = int(1200*(self.validation_ratio) // self.config.params_batch_size)
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2024-04-05 03:59:39,854: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-05 03:59:39,857: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-04-05 03:59:39,858: INFO: common: created directory at: artifacts:]
[2024-04-05 03:59:39,858: INFO: common: created directory at: artifacts\training:]
Found 1200 files belonging to 4 classes.
Using 960 files for training.
Found 1200 files belonging to 4 classes.
Using 240 files for validation.
60/60 ━━━━━━━━━━━━━━━━━━━━ 619s 10s/step - accuracy: 0.2523 - loss: nan - val_accuracy: 0.2250 - val_loss: nan
